<a href="https://colab.research.google.com/github/dyjdlopez/fund_opts_python/blob/main/metaheuristics/metaopts_01_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Constrained optimization studies decision problems in which an objective
function is optimized subject to explicit constraints on the decision
variables. A generic constrained optimization problem can be written as
$$
\begin{aligned}
\min_{x \in \mathbf{R}^n} \quad & f_0(x) \\
\text{s.t.} \quad & f_i(x) \le 0, \quad i = 1,\dots,m, \\
                  & g_j(x) = 0, \quad j = 1,\dots,p,
\end{aligned}
$$
where $x$ is the decision vector, $f_0$ is the objective function,
and the constraint functions $f_i, g_j$ encode limits such as resource
capacities, physical laws, or policy requirements.

In many applications, the constraints are linear, leading to the
widely used form
$$
\begin{aligned}
\min_{x \in \mathbf{R}^n} \quad & f_0(x) \\
\text{s.t.} \quad & A x = b, \\
                  & C x \le d,
\end{aligned}
$$
where $A \in \mathbf{R}^{m_e \times n}$, $b \in \mathbf{R}^{m_e}$,
$C \in \mathbf{R}^{m_i \times n}$, and $d \in \mathbf{R}^{m_i}$
compactly represent systems of equality and inequality constraints.

CVXPY is an open source Python-embedded modeling language tailored for
convex optimization problems. Problems written in mathematical notation like
$$
\min_{x} \; f_0(x)
\quad \text{s.t.} \quad
A x = b,\; C x \le d
$$
are expressed in CVXPY using high-level objects for variables, objectives,
and constraints that closely mirror this algebraic structure CVXPY
verifies convexity using disciplined convex programming rules and
automatically rewrites the problem into a cone program that is passed to
a numerical solver, allowing users to focus on modeling rather than
low-level algorithmic details.


CVXPY is an open-source, Python-embedded modeling language for *convex*
optimization problems, designed so that code closely follows the
mathematical formulation of an optimization model. See the documentation
at [https://www.cvxpy.org/](https://www.cvxpy.org/).

Instead of writing problems in the low-level “standard form” required by
solvers, users declare variables, objectives, and constraints using
high-level linear-algebra syntax (for example, `x = cp.Variable()`,
`A @ x <= b`), and then call `prob.solve()` to obtain a solution.

Internally, CVXPY uses the [disciplined convex programming (DCP)](https://www.cvxpy.org/tutorial/dcp/index.html) ruleset to
verify that a problem is convex and, when it is, automatically rewrites the
user's specification into an equivalent cone program that can be passed to
numerical solvers. This separation between modeling and
numerical algorithms allows users to focus on expressing constrained
optimization problems clearly—such as linear programs, quadratic programs,
and many regularized estimation problems—without needing to implement or
tune specific optimization algorithms themselves.

In [ ]:
import cvxpy as cp

## 1. Solving LP Problems with CVXPY



As an example we will make use of the "Linear programming example 1997 UG exam" from this [source](https://people.brunel.ac.uk/~mastjjb/jeb/or/morelp.html):

> A company makes two products (X and Y) using two machines (A and B). Each unit of X that is produced requires 50 minutes processing time on machine A and 30 minutes processing time on machine B. Each unit of Y that is produced requires 24 minutes processing time on machine A and 33 minutes processing time on machine B.
>
> At the start of the current week there are 30 units of X and 90 units of Y in stock. Available processing time on machine A is forecast to be 40 hours and on machine B is forecast to be 35 hours.
>
> The demand for X in the current week is forecast to be 75 units and for Y is forecast to be 95 units. Company policy is to **maximize** the combined sum of the units of X and the units of Y in stock at the end of the week.


*Let:*

* $x$ be the number of units of X produced in the current week
* $y$ be the number of units of Y produced in the current week

such that:

* $50x + 24y \leq 40 \cdot 60$ *machine A time in mins*
* $30x + 33y \leq 35 \cdot 60$ *machine B time in mins*
* $x \geq 75 - 30$ *production requirement for A*
* $y \geq 95 - 90$ *production requirement for B*

wherein the objective function is set as:
$$
\max_{x, y} \; (30 + x - 75) + (90 + y - 95)
= \max_{x, y} \; (x + y - 50).
$$

### 1.1 Variables

In the LP example, the decision variables are the weekly production
quantities of products X and Y. Mathematically, we can write a decision
vector
$$
p = \begin{bmatrix} x \\ y \end{bmatrix}
\in \mathbf{R}^2,
$$
where $x$ is the number of units of X produced and $y$ is the
number of units of Y produced. In CVXPY, such decision variables are
declared using the `cp.Variable` class.

For this problem, there are a few natural ways to define the variables:

In [ ]:
x = cp.Variable(nonneg=True)
y = cp.Variable(nonneg=True)

Here `nonneg=True` encodes the implicit non-negativity constraint $x\geq 0$ and $y \geq 0$, since production quantities cannot be negative.

In general, CVXPY variables can be:
* Scalars: `cp.Variable()`
* Vectors: `cp.Varaible(n)`
* Matrices: `cp.Variable((m,n))`
with optional keywords such as `nonneg=True`, `boolean=True` or `integer=True` to impose domain restrictions on the decision variables.

### 1.2 Constraints

From the problem data, the linear constraints can be written as:

- Machine A time $\text(Eq. 1)$:
  $$
  50x + 24y \leq 40 \cdot 60
  $$
- Machine B time $\text(Eq. 2)$:
  $$
  30x + 33y \leq 35 \cdot 60
  $$
- Production requirement for X $\text(Eq. 3)$:
  $$
  x \geq 75 - 30
  $$
- Production requirement for Y $\text(Eq. 4)$:
  $$
  y \geq 95 - 90
  $$

In CVXPY, each of these becomes a linear constraint object created by
using `<=` or `>=` between expressions. Assuming scalar variables
`x` and `y` for products X and Y:

In [ ]:
constraints = []

constraints.append(50*x + 24*y <= 40*60) # Eq. 1
constraints.append(30*x + 33*y <= 35*60) # Eq. 2
constraints.append(x >= 75- 30) # Eq. 3
constraints.append(y >= 95- 90) # Eq. 3

### 1.3 Problem

The company's goal is to **maximize** the total end-of-week stock of
products X and Y. End-of-week stock is
$$
\text{stock}_X^{\text{end}} = 30 + x - 75
$$
$$
\text{stock}_Y^{\text{end}} = 90 + y - 95
$$
so the objective function is
$$
\max_{x, y} \; (30 + x - 75) + (90 + y - 95)
= \max_{x, y} \; (x + y - 50).
$$
Equivalently, one can maximize $x + y$, since subtracting the
constant 50 does not change the optimizer.

In CVXPY, the objective and problem are written as:


In [ ]:
objective = cp.Maximize(x + y - 50)

Now let's have a look how an entire pipeline from variable declaration to simulation would look like:

In [ ]:
# Decision variables
x = cp.Variable(nonneg=True)
y = cp.Variable(nonneg=True)

# Constraints
constraints = []

constraints.append(50*x + 24*y <= 40*60) # Eq. 1
constraints.append(30*x + 33*y <= 35*60) # Eq. 2
constraints.append(x >= 75- 30) # Eq. 3
constraints.append(y >= 95- 90) # Eq. 3

# Objective
objective = cp.Maximize(x + y - 50)

# Define and solve the problem
prob = cp.Problem(objective, constraints)
opt_value = prob.solve()

Here:

* `cp.Maximize(...)` encodes the maximization objective,
* `constraints` is the list of linear constraints defined earlier, and
* `cp.Problem(objective, constraints)` forms the full mathematical
optimization problem that CVXPY passes to a suitable solver.

Similar to the actual solution from the source, our answer seems to check out.

In [ ]:
print(f"Optimal value: {opt_value:.3f}")
print(f"Optimal production of X: {x.value:.3f}")
print(f"Optimal production of Y: {y.value:.3f}")

Optimal value: 1.250
Optimal production of X: 45.000
Optimal production of Y: 6.250


### Activity 1
Identify the variables, objecitve function, and constraints of the model. Encode this LP problem and solve using CVXPY:
> A manufacturer produces three types of plastic fixtures. The table below shows the
times required for molding, trimming, and packaging. (Times are in hours per dozen
fixtures, and profits are in dollars per dozen fixtures.)
$$
\begin{array}{c|c|c|c}
\text{Process} & \text{Type A} & \text{Type B} & \text{Type C} \\
\hline
\text{Molding}   & 1           & 2           & \tfrac{3}{2} \\
\text{Trimming}  & \tfrac{2}{3} & \tfrac{2}{3} & 1 \\
\text{Packaging} & \tfrac{1}{2} & \tfrac{1}{3} & \tfrac{1}{2} \\
\text{Profit}    & \$11        & \$16        & \$15 \\
\end{array}
$$
> The maximum amounts of production time that the manufacturer can allocate to each
component are listed below.
> * Molding: 12,000 hours
> * Trimming: 4,600 hours
> * Packaging: 2,400 hours
>
>How many dozen units of each type of fixture should the manufacturer produce to
obtain a maximum profit?

*Answer*: The maximum profit is $100,200, obtained by producing 600 dozen units of Type A, 5100 dozen units of Type B, and 800 dozen units of Type C.

*Write your expressions here*


In [ ]:
### Encode your model here

### Activity 2
Identify the variables, objecitve function, and constraints of the model. Encode this LP problem and solve using CVXPY:
> The liquid portion of a diet is to provide at least 300 calories, 36 units of vitamin A,
and 90 units of vitamin C daily. A cup of dietary drink X provides 60 calories, 12 units
of vitamin A, and 10 units of vitamin C. A cup of dietary drink Y provides 60 calories,
6 units of vitamin A, and 30 units of vitamin C.
>
> Now, assume that dietary drink X costs
\$ 0.12 per cup and drink Y costs \$ 0.15 per cup. How many cups of each drink should be
consumed each day to minimize the cost and still meet the daily requirements?

*Answer*: The minimum cost is $0.66 per day, and this occurs when three cups of drink X and
two cups of drink Y are consumed each day.

*Write your expressions here*



In [ ]:
### Encode your model here

### Activity 3
Identify the variables, objecitve function, and constraints of the model. Encode this LP problem and solve using CVXPY:
> A petroleum company owns two refineries. Refinery 1 costs \$20,000 per day to operate,
and refinery 2 costs \$25,000 per day to operate. The table shows the numbers of barrels
of each grade of oil the refineries can produce each day.
$$
\begin{array}{c|c|c}
\text{Grade} & \text{Refinery 1} & \text{Refinery 2} \\
\hline
\text{High-grade}   & 400 & 300 \\
\text{Medium-grade}  & 300 & 400 \\
\text{Low-grade} & 200 & 500 \\
\end{array}
$$
> The company has orders totaling 25,000 barrels of high-grade oil, 27,000 barrels of
medium-grade oil, and 30,000 barrels of low-grade oil. How many days should it run
each refinery to minimize its costs and still refine enough oil to meet its orders?

*Answer:* Minimum cost is at $1,750,000 at Refinery 1 operating for 25 days and
Refinery 2 for 50 days.

*Write your expressions here*

In [ ]:
### Encode your model here